In [4]:
import MeCab
text = "吾輩は猫である"
t = MeCab.Tagger("-Ochasen")
result = t.parse(text)
print(result)

吾輩	ワガハイ	吾輩	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
猫	ネコ	猫	名詞-一般		
で	デ	だ	助動詞	特殊・ダ	連用形
ある	アル	ある	助動詞	五段・ラ行アル	基本形
EOS



In [5]:
result

'吾輩\tワガハイ\t吾輩\t名詞-代名詞-一般\t\t\nは\tハ\tは\t助詞-係助詞\t\t\n猫\tネコ\t猫\t名詞-一般\t\t\nで\tデ\tだ\t助動詞\t特殊・ダ\t連用形\nある\tアル\tある\t助動詞\t五段・ラ行アル\t基本形\nEOS\n'

In [6]:
#改行を区切りとして分割
results = result.splitlines()
#EOSの行は対象外
for res in results[:-1]:
    #タブを区切りとして分割
    res_split = res.split("\t")
    print(res_split)

['吾輩', 'ワガハイ', '吾輩', '名詞-代名詞-一般', '', '']
['は', 'ハ', 'は', '助詞-係助詞', '', '']
['猫', 'ネコ', '猫', '名詞-一般', '', '']
['で', 'デ', 'だ', '助動詞', '特殊・ダ', '連用形']
['ある', 'アル', 'ある', '助動詞', '五段・ラ行アル', '基本形']


In [16]:
documents = ["子供が走る","車が走る","子供の脇を車が走る","子供の脇を車と子供が走る"]
words_list = []
t = MeCab.Tagger("-Ochasen")

for s in documents:
    s_parsed = t.parse(s)
    words_s = []
    for line in s_parsed.splitlines()[:-1]:
        words_s.append(line.split("\t")[0])
    words_list.append(words_s)
    
print(words_list)

[['子供', 'が', '走る'], ['車', 'が', '走る'], ['子供', 'の', '脇', 'を', '車', 'が', '走る'], ['子供', 'の', '脇', 'を', '車', 'と', '子供', 'が', '走る']]


In [17]:
word2int = {}
i = 0
for words in words_list:
    for word in words:
        if word not in word2int:
            word2int[word] = i
            i+= 1
print(word2int)

{'子供': 0, 'が': 1, '走る': 2, '車': 3, 'の': 4, '脇': 5, 'を': 6, 'と': 7}


In [18]:
import numpy as np
#各文書毎の単語の出現回数を生成
bow = np.zeros((len(words_list),len(word2int)))
for i,words in enumerate(words_list):
    for word in words:
        bow[i,word2int[word]] += 1
print(bow)

[[1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0.]
 [2. 1. 1. 1. 1. 1. 1. 1.]]


In [19]:
import pandas as pd
pd.DataFrame(bow,columns=list(word2int))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


,子供,が,走る,車,の,脇,を,と
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [23]:
from gensim import corpora
word2int_gs = corpora.Dictionary(words_list)
print(word2int_gs)

Dictionary(8 unique tokens: ['が', '子供', '走る', '車', 'の']...)


In [24]:
print(word2int_gs.token2id)

{'が': 0, '子供': 1, '走る': 2, '車': 3, 'の': 4, 'を': 5, '脇': 6, 'と': 7}


In [31]:
#１番目の文書に含まれる単語の出現回数をカウント
print(word2int_gs.doc2bow(words_list[1]))

[(0, 1), (2, 1), (3, 1)]


In [29]:
words_list[3]

['子供', 'の', '脇', 'を', '車', 'と', '子供', 'が', '走る']

In [28]:
words_list

[['子供', 'が', '走る'],
 ['車', 'が', '走る'],
 ['子供', 'の', '脇', 'を', '車', 'が', '走る'],
 ['子供', 'の', '脇', 'を', '車', 'と', '子供', 'が', '走る']]

In [39]:
import numpy as np
from gensim import matutils

bow_gs = np.array(
    [matutils.corpus2dense(
        [word2int_gs.doc2bow(words)],
        num_terms = len(word2int)).T[0]
        for words in words_list]
    ).astype(np.int)

print(bow_gs)

[[1 1 1 0 0 0 0 0]
 [1 0 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 0]
 [1 2 1 1 1 1 1 1]]


In [40]:
bow_gs_df = pd.DataFrame(bow_gs,columns=list(word2int_gs.values()))
bow_gs_df

,が,子供,走る,車,の,を,脇,と
0,1,1,1,0,0,0,0,0
1,1,0,1,1,0,0,0,0
2,1,1,1,1,1,1,1,0
3,1,2,1,1,1,1,1,1


In [48]:
words_split = np.array(['  '.join(words) for words in words_list])
print(words_split)

['子供  が  走る' '車  が  走る' '子供  の  脇  を  車  が  走る'
 '子供  の  脇  を  車  と  子供  が  走る']


In [49]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w + \\b')

bow_vec = vectorizer.fit_transform(words_split)
bow_vec.toarray()

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [1, 0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1]], dtype=int64)

In [50]:
vectorizer.get_feature_names()

['が  ', 'と  ', 'の  ', 'を  ', '脇  ', '車  ']

In [52]:
### TF-IDF ###
bow_gs_df

,が,子供,走る,車,の,を,脇,と
0,1,1,1,0,0,0,0,0
1,1,0,1,1,0,0,0,0
2,1,1,1,1,1,1,1,0
3,1,2,1,1,1,1,1,1


In [53]:
tf = bow_gs
print(tf)

[[1 1 1 0 0 0 0 0]
 [1 0 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 0]
 [1 2 1 1 1 1 1 1]]


In [54]:
idf = np.log((bow_gs.shape[0] + 1)/(np.sum(bow_gs,axis=0,keepdims=0)+1))
print(idf)

[0.         0.         0.         0.22314355 0.51082562 0.51082562
 0.51082562 0.91629073]


In [55]:
tf_idf = tf + (idf + 1)
tf_idf_normarized = tf_idf / np.sqrt(np.sum(tf_idf ** 2,axis=1,keepdims=True))
print(tf_idf_normarized)

[[0.408112   0.408112   0.408112   0.24958978 0.30829303 0.30829303
  0.30829303 0.39103062]
 [0.40437209 0.20218605 0.40437209 0.44948861 0.30546786 0.30546786
  0.30546786 0.38744725]
 [0.31811311 0.31811311 0.31811311 0.35360555 0.39936327 0.39936327
  0.39936327 0.3047986 ]
 [0.28467089 0.42700633 0.28467089 0.31643213 0.35737948 0.35737948
  0.35737948 0.41509154]]


In [57]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,norm = "l2",smooth_idf=True)
print(tfidf.fit_transform(bow_gs).toarray())

[[0.53482206 0.65416415 0.53482206 0.         0.         0.
  0.         0.        ]
 [0.53482206 0.         0.53482206 0.65416415 0.         0.
  0.         0.        ]
 [0.2906198  0.35546974 0.2906198  0.35546974 0.43907584 0.43907584
  0.43907584 0.        ]
 [0.22360482 0.5470016  0.22360482 0.2735008  0.3378279  0.3378279
  0.3378279  0.42849185]]


In [114]:
#極性判定
import zipfile
import urllib.request

# urllib.request.urlretrieve("https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip")
with zipfile.ZipFile("789_ruby_5639.zip","r") as zipf:
    data = zipf.read("wagahaiwa_nekodearu.txt")

In [115]:
import re
text = data.decode("shift_jis")
#ルビ、注釈、改行コード等を除去
text = re.split( r'\-{5,}',text)[2]
text = text.split("底本：")[0]
text = re.sub(r'《.+?》' ,'',text)
text = re.sub(r'［＃.+?］' ,'',text)
text = text.strip()
#空白文字などを除去
text = text .replace("\u3000","")
text = text .replace("\r","") .replace("\n","")
sentences = text.split("。")
print(sentences[:10])

['一吾輩は猫である', '名前はまだ無い', 'どこで生れたかとんと見当がつかぬ', '何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している', '吾輩はここで始めて人間というものを見た', 'しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ', 'この書生というのは時々我々を捕えて煮て食うという話である', 'しかしその当時は何という考もなかったから別段恐しいとも思わなかった', 'ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである', '掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう']


In [116]:
words_list = []
t=MeCab.Tagger("-Ochasen")
for sentence in sentences[:-1]:
    sentence_parsed = t.parse(sentence)
    word_s = []
    for line in sentence_parsed.splitlines()[:-1]:
        word_s.append(line.split('\t')[2])
    words_list.append(word_s)
    
print(words_list[:10])

[['一', '吾輩', 'は', '猫', 'だ', 'ある'], ['名前', 'は', 'まだ', '無い'], ['どこ', 'で', '生れる', 'た', 'か', 'とんと', '見当', 'が', 'つく', 'ぬ'], ['何', 'でも', '薄暗い', 'じめじめ', 'する', 'た', '所', 'で', 'ニャーニャー', '泣く', 'て', 'いた事', 'だけ', 'は', '記憶', 'する', 'て', 'いる'], ['吾輩', 'は', 'ここ', 'で', '始める', 'て', '人間', 'という', 'もの', 'を', '見る', 'た'], ['しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '｜', '獰悪', 'だ', '種族', 'だ', 'ある', 'た', 'そう', 'だ'], ['この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕える', 'て', '煮る', 'て', '食う', 'という', '話', 'だ', 'ある'], ['しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'ない', 'た', 'から', '別段', '恐い', 'いとも', '思う', 'ない', 'た'], ['ただ', '彼', 'の', '掌', 'に', '載せる', 'られる', 'て', 'スー', 'と', '持ち上げる', 'られる', 'た', '時', '何だか', 'フワフワ', 'する', 'た', '感じ', 'が', 'ある', 'た', 'ばかり', 'だ', 'ある'], ['掌', 'の', '上', 'で', '少し', '落ちつく', 'て', '書生', 'の', '顔', 'を', '見る', 'た', 'の', 'が', 'いわゆる', '人間', 'という', 'もの', 'の', '見る', '始', 'だ', 'ある', 'う']]


In [117]:
urllib.request.urlretrieve("http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn","wago.121808.pn")

('wago.121808.pn', <http.client.HTTPMessage at 0x1141ed3c8>)

In [118]:
#日本語評価極性辞書を読み込む
wago = pd.read_csv("wago.121808.pn",header = None,sep = "\t")
wago.head(3)

,0,1
0,ネガ（経験）,あがく
1,ネガ（経験）,あきらめる
2,ネガ（経験）,あきる


In [120]:
word2score = {}
values = {"ポジ（経験）":1,"ポジ（評価）":1,"ネガ（経験）":-1,"ネガ（評価）":-1}
for word,label in zip(wago.loc[:,1],wago.loc[:,0]):
    word2score[word] = values[label]

In [121]:
list(word2score.items())[:3]

[('あがく', -1), ('あきらめる', -1), ('あきる', -1)]

In [124]:
scores = []
for words in words_list:
    score = 0
    for word in words:
        if word in word2score:
            score += word2score[word]
    scores.append(score)

In [128]:
scores_df = pd.DataFrame({"sentence":sentences[:-1],"score":scores},columns=["sentence","score"])
scores_df.head(5)

,sentence,score
0,一吾輩は猫である,0
1,名前はまだ無い,0
2,どこで生れたかとんと見当がつかぬ,0
3,何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している,-1
4,吾輩はここで始めて人間というものを見た,0


In [129]:
scores_df_sorted = scores_df.sort_values("score",ascending=False)
scores_df_sorted.head(5)

,sentence,score
1428,四百六十五行から、四百七十三行を御覧になると分ります」「希臘語｜云々はよした方がいい、さも希...,5
453,「厭きっぽいのじゃない薬が利かんのだ」「それだってせんだってじゅうは大変によく利くよく利くと...,5
5380,精神の修養を主張するところなぞは大に敬服していい」「敬服していいかね,4
3860,美しい？美しくても構わんから、美しい獣と見做せばいいのである,4
3871,それほど裸体がいいものなら娘を裸体にして、ついでに自分も裸になって上野公園を散歩でもするがい...,3


In [130]:
scores_df_sorted.tail(5)

,sentence,score
7014,自殺クラブはこの第二の問題と共に起るべき運命を有している」「なるほど」「死ぬ事は苦しい、しか...,-3
7098,向うがあやまるなら特別、私の方ではそんな慾はありません」「警察が君にあやまれと命じたらどうで...,-4
3783,こんな、しつこい、毒悪な、ねちねちした、執念深い奴は大嫌だ,-4
6618,どうもいつまで行っても柿ばかり食ってて際限がないね」「私もじれったくてね」「君より聞いてる方...,-4
6687,「古人を待つ身につらき置炬燵と云われた事があるからね、また待たるる身より待つ身はつらいともあ...,-5
